In [2]:
import  pandas      as pd
import  requests
import  calendar
import  json

from    sqlalchemy  import create_engine
from    bs4         import BeautifulSoup
import  configparser


In [4]:
config = configparser.ConfigParser()
config.read('../../my_config.ini')
 
# get all the connections
config.sections()
DBPass = config.get('postgres', 'password')
FlightKey = config.get('FlightLabs', 'mykey')


In [5]:
airport_file = "../Resources/list-of-airports-in-australia-hxl-tags-1.csv"
airport_df = pd.read_csv(airport_file,skiprows=[1])
airport_df.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords,score,last_updated
0,27145,YSSY,large_airport,Sydney Kingsford Smith International Airport,-33.946098,151.177002,21.0,OC,AU,AU-NSW,Sydney,1,YSSY,SYD,NaN,http://www.sydneyairport.com.au/,http://en.wikipedia.org/wiki/Kingsford_Smith_I...,RAAF Station Mascot,1027975,2008-06-26T14:52:34+00:00
1,27066,YMML,large_airport,Melbourne International Airport,-37.673302,144.843002,434.0,OC,AU,AU-VIC,Melbourne,1,YMML,MEL,NaN,http://melbourneairport.com.au/,http://en.wikipedia.org/wiki/Melbourne_Airport,NaN,1016075,2017-03-06T19:29:22+00:00
2,26901,YBBN,large_airport,Brisbane International Airport,-27.384199,153.117004,13.0,OC,AU,AU-QLD,Brisbane,1,YBBN,BNE,NaN,NaN,http://en.wikipedia.org/wiki/Brisbane_Airport,NaN,1013175,2008-06-26T14:52:34+00:00
3,27119,YPPH,large_airport,Perth International Airport,-31.940300,115.967003,67.0,OC,AU,AU-WA,Perth,1,YPPH,PER,NaN,NaN,http://en.wikipedia.org/wiki/Perth_Airport,NaN,51375,2008-06-26T14:52:34+00:00
4,26904,YBCS,medium_airport,Cairns International Airport,-16.885799,145.755005,10.0,OC,AU,AU-QLD,Cairns,1,YBCS,CNS,NaN,NaN,http://en.wikipedia.org/wiki/Cairns_Internatio...,NaN,1050,2010-08-29T23:17:14+00:00


In [4]:
formatted_airport_df = airport_df[['id', 'ident', 'type','name','latitude_deg','longitude_deg','iso_region','municipality','gps_code','iata_code','score']].copy()
formatted_airport_df.head()


,id,ident,type,name,latitude_deg,longitude_deg,iso_region,municipality,gps_code,iata_code,score
0,27145,YSSY,large_airport,Sydney Kingsford Smith International Airport,-33.946098,151.177002,AU-NSW,Sydney,YSSY,SYD,1027975
1,27066,YMML,large_airport,Melbourne International Airport,-37.673302,144.843002,AU-VIC,Melbourne,YMML,MEL,1016075
2,26901,YBBN,large_airport,Brisbane International Airport,-27.384199,153.117004,AU-QLD,Brisbane,YBBN,BNE,1013175
3,27119,YPPH,large_airport,Perth International Airport,-31.940300,115.967003,AU-WA,Perth,YPPH,PER,51375
4,26904,YBCS,medium_airport,Cairns International Airport,-16.885799,145.755005,AU-QLD,Cairns,YBCS,CNS,1050


In [5]:
#Rename columns
rename_dict = {'latitude_deg':'latitude',
               'longitude_deg':'longitude',
               'iso_region':'region'

}

formatted_airport_df.rename(columns = rename_dict, inplace = True)

#Process Region
formatted_airport_df['region'] = formatted_airport_df['region'].str[3:9]

In [6]:
formatted_airport_df.head()

,id,ident,type,name,latitude,longitude,region,municipality,gps_code,iata_code,score
0,27145,YSSY,large_airport,Sydney Kingsford Smith International Airport,-33.946098,151.177002,NSW,Sydney,YSSY,SYD,1027975
1,27066,YMML,large_airport,Melbourne International Airport,-37.673302,144.843002,VIC,Melbourne,YMML,MEL,1016075
2,26901,YBBN,large_airport,Brisbane International Airport,-27.384199,153.117004,QLD,Brisbane,YBBN,BNE,1013175
3,27119,YPPH,large_airport,Perth International Airport,-31.940300,115.967003,WA,Perth,YPPH,PER,51375
4,26904,YBCS,medium_airport,Cairns International Airport,-16.885799,145.755005,QLD,Cairns,YBCS,CNS,1050


In [6]:
airport_traffic = "../Resources/TopRoutesJuly2014May2022.xlsx"

range1 = [i for i in range(0,11)]
range2 = [i for i in range(12,15)]
range3 = [i for i in range(16,19)]
range4 = [i for i in range(20,21)]

usecols = range1 + range2 + range3 + range4
skiprows = [i for i in range(1,13)]


#airport_traffic_df = pd.read_excel(airport_traffic, sheet_name='Top Routes',skiprows=[1,2,3,4,5,6,7,8,9,10,11,12],header=[1],usecols=usecols)
airport_traffic_df = pd.read_excel(airport_traffic, sheet_name='Top Routes',skiprows=skiprows,header=[1],usecols=usecols)

rename_dict = {}
#Rename columns
rename_dict = {airport_traffic_df.columns[1]:'Airport_Code_1',
               airport_traffic_df.columns[0]:'Airport_Code_0',
               airport_traffic_df.columns[2]:'Year',
               airport_traffic_df.columns[3]:'Month',
               airport_traffic_df.columns[4]:'Revenue_Passenger', 
               airport_traffic_df.columns[5]:'Aircraft_Trips',
               airport_traffic_df.columns[6]:'Rev_Pax_LF_Perc',
               airport_traffic_df.columns[7]:'Distance_klm',
               airport_traffic_df.columns[8]:'RPKs',
               airport_traffic_df.columns[9]:'ASKs',
               airport_traffic_df.columns[10]:'Seats',
               airport_traffic_df.columns[11]:'Annual_Revenue_Passenger', 
               airport_traffic_df.columns[12]:'Annual_Aircraft_Trips',
               airport_traffic_df.columns[13]:'Annual_Rev_Pax_LF_Perc',
               airport_traffic_df.columns[14]:'Annual_RPKs',
               airport_traffic_df.columns[15]:'Annual_ASKs',
               airport_traffic_df.columns[16]:'Annual_Seats',
               airport_traffic_df.columns[17]:'City_Pair_Route'
}

airport_traffic_df.rename(columns = rename_dict, inplace = True)
airport_traffic_df.head()


,Airport_Code_0,Airport_Code_1,Year,Month,Revenue_Passenger,Aircraft_Trips,Rev_Pax_LF_Perc,Distance_klm,RPKs,ASKs,Seats,Annual_Revenue_Passenger,Annual_Aircraft_Trips,Annual_Rev_Pax_LF_Perc,Annual_RPKs,Annual_ASKs,Annual_Seats,City_Pair_Route
0,ABX,SYD,2014,7,19747,589.0,65.807978,452.0,8925644.0,13563164.0,30007.0,229468,6600.0,68.588611,103719536.0,151219764.0,334557.0,Albury - Sydney
1,ABX,SYD,2014,8,21085,573.0,69.183319,452.0,9530420.0,13775604.0,30477.0,229262,6581.0,68.368865,103626424.0,151569612.0,335331.0,Albury - Sydney
2,ABX,SYD,2014,9,20138,536.0,67.486595,452.0,9102376.0,13487680.0,29840.0,228841,6553.0,67.894663,103436132.0,152347956.0,337053.0,Albury - Sydney
3,ABX,SYD,2014,10,20620,545.0,67.907130,452.0,9320240.0,13724980.0,30365.0,228101,6520.0,67.366714,103101652.0,153045392.0,338596.0,Albury - Sydney
4,ABX,SYD,2014,11,19516,508.0,69.166430,452.0,8821232.0,12753632.0,28216.0,227120,6464.0,67.096606,102658240.0,153000644.0,338497.0,Albury - Sydney


In [8]:
airport_traffic_df = airport_traffic_df.dropna()

In [9]:
dtypes_list = {}
dtypes_list = {"Aircraft_Trips":'int',
               "Distance_klm":'int',
               "RPKs":'int', 
               "ASKs":'int', 
               "Seats":'int', 
               "Annual_Aircraft_Trips":'int',
               "Annual_RPKs":'int',
               "Annual_ASKs":'int',
               "Annual_Seats":'int'
               }


airport_traffic_df = airport_traffic_df.astype(dtypes_list) 
airport_traffic_df.head()


,Airport_Code_0,Airport_Code_1,Year,Month,Revenue_Passenger,Aircraft_Trips,Rev_Pax_LF_Perc,Distance_klm,RPKs,ASKs,Seats,Annual_Revenue_Passenger,Annual_Aircraft_Trips,Annual_Rev_Pax_LF_Perc,Annual_RPKs,Annual_ASKs,Annual_Seats,City_Pair_Route
0,ABX,SYD,2014,7,19747,589,65.807978,452,8925644,13563164,30007,229468,6600,68.588611,103719536,151219764,334557,Albury - Sydney
1,ABX,SYD,2014,8,21085,573,69.183319,452,9530420,13775604,30477,229262,6581,68.368865,103626424,151569612,335331,Albury - Sydney
2,ABX,SYD,2014,9,20138,536,67.486595,452,9102376,13487680,29840,228841,6553,67.894663,103436132,152347956,337053,Albury - Sydney
3,ABX,SYD,2014,10,20620,545,67.907130,452,9320240,13724980,30365,228101,6520,67.366714,103101652,153045392,338596,Albury - Sydney
4,ABX,SYD,2014,11,19516,508,69.166430,452,8821232,12753632,28216,227120,6464,67.096606,102658240,153000644,338497,Albury - Sydney


In [10]:
#Format the Month name
airport_traffic_df['MM']    = airport_traffic_df['Month'] 
airport_traffic_df['Month'] = airport_traffic_df['Month'].apply(lambda x: calendar.month_abbr[x])
airport_traffic_df.head()

,Airport_Code_0,Airport_Code_1,Year,Month,Revenue_Passenger,Aircraft_Trips,Rev_Pax_LF_Perc,Distance_klm,RPKs,ASKs,Seats,Annual_Revenue_Passenger,Annual_Aircraft_Trips,Annual_Rev_Pax_LF_Perc,Annual_RPKs,Annual_ASKs,Annual_Seats,City_Pair_Route,MM
0,ABX,SYD,2014,Jul,19747,589,65.807978,452,8925644,13563164,30007,229468,6600,68.588611,103719536,151219764,334557,Albury - Sydney,7
1,ABX,SYD,2014,Aug,21085,573,69.183319,452,9530420,13775604,30477,229262,6581,68.368865,103626424,151569612,335331,Albury - Sydney,8
2,ABX,SYD,2014,Sep,20138,536,67.486595,452,9102376,13487680,29840,228841,6553,67.894663,103436132,152347956,337053,Albury - Sydney,9
3,ABX,SYD,2014,Oct,20620,545,67.907130,452,9320240,13724980,30365,228101,6520,67.366714,103101652,153045392,338596,Albury - Sydney,10
4,ABX,SYD,2014,Nov,19516,508,69.166430,452,8821232,12753632,28216,227120,6464,67.096606,102658240,153000644,338497,Albury - Sydney,11


In [11]:
airport_traffic_df.dtypes

Airport_Code_0               object
Airport_Code_1               object
Year                          int64
Month                        object
Revenue_Passenger            object
Aircraft_Trips                int32
Rev_Pax_LF_Perc             float64
Distance_klm                  int32
RPKs                          int32
ASKs                          int32
Seats                         int32
Annual_Revenue_Passenger     object
Annual_Aircraft_Trips         int32
Annual_Rev_Pax_LF_Perc      float64
Annual_RPKs                   int32
Annual_ASKs                   int32
Annual_Seats                  int32
City_Pair_Route              object
MM                            int64
dtype: object

In [7]:
World_city_file = "../Resources/World Cities.json"
world_city_df = pd.read_json(World_city_file)
world_city_df.head()

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Sydney,-33.8650,151.2094,Australia,AU,New South Wales,admin,5312163,4840600
1,Melbourne,-37.8136,144.9631,Australia,AU,Victoria,admin,5078193,4529500
2,Brisbane,-27.4678,153.0281,Australia,AU,Queensland,admin,2514184,2360241
3,Perth,-31.9522,115.8589,Australia,AU,Western Australia,admin,2059484,2039200
4,Adelaide,-34.9289,138.6011,Australia,AU,South Australia,admin,1345777,1295714


In [13]:
au_city_df = world_city_df[(world_city_df.iso2 == "AU")].reset_index(drop=True)
au_city_df['city']       = au_city_df['city'].str.upper() 
au_city_df.rename(columns = {'admin_name':'state'}, inplace = True)
au_city_df.head()

,city,lat,lng,country,iso2,state,capital,population,population_proper
0,SYDNEY,-33.8650,151.2094,Australia,AU,New South Wales,admin,5312163,4840600
1,MELBOURNE,-37.8136,144.9631,Australia,AU,Victoria,admin,5078193,4529500
2,BRISBANE,-27.4678,153.0281,Australia,AU,Queensland,admin,2514184,2360241
3,PERTH,-31.9522,115.8589,Australia,AU,Western Australia,admin,2059484,2039200
4,ADELAIDE,-34.9289,138.6011,Australia,AU,South Australia,admin,1345777,1295714


In [14]:
wikiurl="https://en.wikipedia.org/wiki/States_and_territories_of_Australia"

table_class="wikitable sortable" #From the page inspect


response=requests.get(wikiurl)
print(response.status_code)

soup = BeautifulSoup(response.text, 'html.parser')

indiatable=soup.find_all('table',{'class':"wikitable sortable"})
#print(soup.prettify())


states_list=pd.read_html(str(indiatable))

#First Table in the Wiki
#print(states_list[0])

#Second Table in the Wiki
#print(states_list[1])


200


In [15]:
states_au_df = pd.DataFrame(states_list[0])
states_au_df = pd.DataFrame(states_list[0])
states_au_df = states_au_df[['State','Postal','Capital','Population(Dec 2021)[6]', 'Area (km2)[7]']]
states_au_df.rename(columns = {'State':'State_or_Territory'}, inplace = True)
states_au_df.head()

,State_or_Territory,Postal,Capital,Population(Dec 2021)[6],Area (km2)[7]
0,New South Wales,NSW,Sydney,8095430,809952
1,Victoria,VIC,Melbourne,6559941,237657
2,Queensland,QLD,Brisbane,5265043,1851736
3,Western Australia,WA,Perth[c],2762234,2642753
4,South Australia,SA,Adelaide,1806599,1044353


In [16]:
states_external = pd.DataFrame(states_list[1])
states_external = pd.DataFrame(states_list[1])
states_external = states_external[['Territory','Postal','Capital(or largest settlement)','Population(Dec 2021)[6]', 'Area (km2)[7]']]

states_external.rename(columns = {'Territory':'State_or_Territory', 'Capital(or largest settlement)':'Capital'}, inplace = True)
states_external.head()


,State_or_Territory,Postal,Capital,Population(Dec 2021)[6],Area (km2)[7]
0,Australian Capital Territory,ACT,Canberra,453324,2358
1,Northern Territory,NT,Darwin,249345,1419630
2,Jervis Bay Territory,JBT,None(Jervis Bay Village),405,67


In [17]:
formatted_states_df = pd.merge(states_au_df,states_external, how ='outer' )
formatted_states_df.rename(columns = {'Postal':'state_code','Population(Dec 2021)[6]':'Population_Dec_2021', 'Area (km2)[7]':'Area_km2'}, inplace = True)

print(formatted_states_df)

             State_or_Territory state_code                   Capital  \
0               New South Wales        NSW                    Sydney   
1                      Victoria        VIC                 Melbourne   
2                    Queensland        QLD                  Brisbane   
3             Western Australia         WA                  Perth[c]   
4               South Australia         SA                  Adelaide   
5                      Tasmania        TAS                    Hobart   
6  Australian Capital Territory        ACT                  Canberra   
7            Northern Territory         NT                    Darwin   
8          Jervis Bay Territory        JBT  None(Jervis Bay Village)   

   Population_Dec_2021  Area_km2  
0              8095430    809952  
1              6559941    237657  
2              5265043   1851736  
3              2762234   2642753  
4              1806599   1044353  
5               569827     90758  
6               453324      2358  

In [18]:
au_city_df = pd.merge(au_city_df, formatted_states_df, left_on=['state'], right_on=['State_or_Territory'] ) #, how="outer")
au_city_df.head()

,city,lat,lng,country,iso2,state,capital,population,population_proper,State_or_Territory,state_code,Capital,Population_Dec_2021,Area_km2
0,SYDNEY,-33.8650,151.2094,Australia,AU,New South Wales,admin,5312163,4840600,New South Wales,NSW,Sydney,8095430,809952
1,NEWCASTLE,-32.9167,151.7500,Australia,AU,New South Wales,,322278,152984,New South Wales,NSW,Sydney,8095430,809952
2,WOLLONGONG,-34.4331,150.8831,Australia,AU,New South Wales,,302739,289236,New South Wales,NSW,Sydney,8095430,809952
3,MAITLAND,-32.7167,151.5500,Australia,AU,New South Wales,,78015,1885,New South Wales,NSW,Sydney,8095430,809952
4,COFFS HARBOUR,-30.3022,153.1189,Australia,AU,New South Wales,,71822,25752,New South Wales,NSW,Sydney,8095430,809952


In [19]:
formatted_city_df = au_city_df[['city','lat','lng','state_code','population']]
formatted_city_df.head()

,city,lat,lng,state_code,population
0,SYDNEY,-33.8650,151.2094,NSW,5312163
1,NEWCASTLE,-32.9167,151.7500,NSW,322278
2,WOLLONGONG,-34.4331,150.8831,NSW,302739
3,MAITLAND,-32.7167,151.5500,NSW,78015
4,COFFS HARBOUR,-30.3022,153.1189,NSW,71822


In [8]:
roads_file = "../Resources/ardd_fatalities_jun2022.xlsx"

skiprows = [i for i in range(0,4)]

roads_file_df = pd.read_excel(roads_file, sheet_name='BITRE_Fatality',skiprows=skiprows)
roads_file_df.rename(columns = {'Heavy Rigid Truck Involvement':'Heavy Rigid Truck Inv', 'Articulated Truck Involvement':'Articulated Truck Inv'}, inplace = True)

roads_file_df.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Bus Involvement,Heavy Rigid Truck Inv,Articulated Truck Inv,...,Age,National Remoteness Areas,SA4 Name 2016,National LGA Name 2017,National Road Type,Christmas Period,Easter Period,Age Group,Day of week,Time of day
0,20222043,Vic,6,2022,Saturday,23:25:00,Multiple,-9,-9,-9,...,29,Major Cities of Australia,Melbourne - West,Wyndham (C),Local Road,No,No,26_to_39,Weekend,Night
1,20225014,WA,6,2022,Friday,18:47:00,Single,No,No,No,...,40,NaN,NaN,NaN,NaN,No,No,40_to_64,Weekend,Night
2,20227008,NT,6,2022,Saturday,-9,-9,-9,-9,-9,...,-9,NaN,NaN,NaN,NaN,No,No,-9,Weekend,Night
3,20227013,NT,6,2022,Tuesday,-9,-9,-9,-9,-9,...,-9,NaN,NaN,NaN,NaN,No,No,-9,Weekday,Night
4,20222114,Vic,6,2022,Sunday,01:18:00,Multiple,-9,-9,-9,...,18,Inner Regional Australia,Warrnambool and South West,Corangamite (S),Local Road,No,No,17_to_25,Weekend,Night


In [21]:
roads_file_df.columns

Index(['Crash ID', 'State', 'Month', 'Year', 'Dayweek', 'Time', 'Crash Type',
       'Bus Involvement', 'Heavy Rigid Truck Inv', 'Articulated Truck Inv',
       'Speed Limit', 'Road User', 'Gender', 'Age',
       'National Remoteness Areas', 'SA4 Name 2016', 'National LGA Name 2017',
       'National Road Type', 'Christmas Period', 'Easter Period', 'Age Group',
       'Day of week', 'Time of day'],
      dtype='object')

In [22]:
roads_file_df['MM']    = roads_file_df['Month'] 
roads_file_df['Month'] = roads_file_df['Month'].apply(lambda x: calendar.month_abbr[x])
roads_file_df['State'] = roads_file_df['State'].str.upper() 
roads_file_df.head()

,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Bus Involvement,Heavy Rigid Truck Inv,Articulated Truck Inv,...,National Remoteness Areas,SA4 Name 2016,National LGA Name 2017,National Road Type,Christmas Period,Easter Period,Age Group,Day of week,Time of day,MM
0,20222043,VIC,Jun,2022,Saturday,23:25:00,Multiple,-9,-9,-9,...,Major Cities of Australia,Melbourne - West,Wyndham (C),Local Road,No,No,26_to_39,Weekend,Night,6
1,20225014,WA,Jun,2022,Friday,18:47:00,Single,No,No,No,...,NaN,NaN,NaN,NaN,No,No,40_to_64,Weekend,Night,6
2,20227008,NT,Jun,2022,Saturday,-9,-9,-9,-9,-9,...,NaN,NaN,NaN,NaN,No,No,-9,Weekend,Night,6
3,20227013,NT,Jun,2022,Tuesday,-9,-9,-9,-9,-9,...,NaN,NaN,NaN,NaN,No,No,-9,Weekday,Night,6
4,20222114,VIC,Jun,2022,Sunday,01:18:00,Multiple,-9,-9,-9,...,Inner Regional Australia,Warrnambool and South West,Corangamite (S),Local Road,No,No,17_to_25,Weekend,Night,6


In [23]:
rds_connection_string = f"postgres:{print(DBPass)}@localhost:5432/project2"
engine = create_engine(f'postgresql://{rds_connection_string}')

hgfh


In [24]:
engine.table_names()

c:\Users\User\anaconda3\envs\bootcamp\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['city', 'airports', 'airport_traffic', 'states', 'road_fatalities', 'flights']

In [25]:
formatted_city_df.to_sql(name='city', con=engine, if_exists='replace', index=False)

In [26]:
formatted_airport_df.to_sql(name='airports', con=engine, if_exists='replace', index=False)

In [27]:
airport_traffic_df.to_sql(name='airport_traffic', con=engine, if_exists='replace', index=False)

In [28]:
formatted_states_df.to_sql(name='states', con=engine, if_exists='replace', index=False)

In [29]:
roads_file_df.to_sql(name='road_fatalities', con=engine, if_exists='replace', index=False)



In [30]:
formatted_airport_df.head()

list_airport_df = formatted_airport_df[(formatted_airport_df.type == 'large_airport') ] 
list_airport_df.head()


,id,ident,type,name,latitude,longitude,region,municipality,gps_code,iata_code,score
0,27145,YSSY,large_airport,Sydney Kingsford Smith International Airport,-33.946098,151.177002,NSW,Sydney,YSSY,SYD,1027975
1,27066,YMML,large_airport,Melbourne International Airport,-37.673302,144.843002,VIC,Melbourne,YMML,MEL,1016075
2,26901,YBBN,large_airport,Brisbane International Airport,-27.384199,153.117004,QLD,Brisbane,YBBN,BNE,1013175
3,27119,YPPH,large_airport,Perth International Airport,-31.940300,115.967003,WA,Perth,YPPH,PER,51375
5,27096,YPAD,large_airport,Adelaide International Airport,-34.945000,138.531006,SA,Adelaide,YPAD,ADL,975


In [31]:
for index, row in list_airport_df.iloc[:1].iterrows():    
    print(row)

id                                                     27145
ident                                                   YSSY
type                                           large_airport
name            Sydney Kingsford Smith International Airport
latitude                                          -33.946098
longitude                                         151.177002
region                                                   NSW
municipality                                          Sydney
gps_code                                                YSSY
iata_code                                                SYD
score                                                1027975
Name: 0, dtype: object


In [40]:
flight_Date = []

departure_iata  = []
dept_terminal   = []
dept_gate       = []
delay           = []
dept_scheduled  = []

arr_iata        = []
arr_terminal    = []
arr_gate        = []
arr_baggage     = []
airline         = []
airline_iata    = []
flight_iata     = []


base_url = "https://app.goflightlabs.com/flights?"
for index, row in list_airport_df.iterrows(): 
    #continue   


    query_url = f"{base_url}access_key={FlightKey}&flight_status=active&dep_iata={row.iata_code}"


    try:
            response = requests.get(query_url)   
            places_data = response.json()


            if response.status_code == 200:


                for a in places_data:

                    flight_Date.append(a['flight_date'])
                    departure_iata.append(a['departure']['iata'])
                    dept_terminal.append(a['departure']['terminal'])
                    dept_gate.append(a['departure']['gate'])
                    delay.append(a['departure']['delay'])
                    dept_scheduled.append(a['departure']['scheduled'])

                    arr_iata.append(a['arrival']['iata'])
                    arr_terminal.append(a['arrival']['terminal'])
                    arr_gate.append(a['arrival']['gate'])
                    arr_baggage.append(a['arrival']['baggage'])
                    airline.append(a['airline']['name'])
                    airline_iata.append(a['airline']['iata'])
                    flight_iata.append(a['flight']['iata'])


    except KeyError:
        print(f"Unexpected Error calling API") 


flight_df = pd.DataFrame({            
                        'flight_Date':flight_Date,            
                        'departure_iata':departure_iata,
                        'dept_terminal':dept_terminal,             
                        'dept_gate': dept_gate,
                        'delay':delay,

                        'arr_iata':arr_iata,            
                        'arr_terminal':arr_terminal,
                        'arr_gate':arr_gate,             
                        'arr_baggage': arr_baggage,

                        'airline':airline,            
                        'airline_iata':airline_iata,
                        'flight_iata':flight_iata             
                    })

In [41]:
flight_df.to_sql(name='flights', con=engine, if_exists='append', index=False)

In [42]:
flight_df.head()

,flight_Date,departure_iata,dept_terminal,dept_gate,delay,arr_iata,arr_terminal,arr_gate,arr_baggage,airline,airline_iata,flight_iata
0,2022-08-01,SYD,1,33,46.0,BKK,2E,None,None,El Al,LY,LY8780
1,2022-08-01,SYD,1,33,46.0,BKK,2E,None,None,KLM,KL,KL4893
2,2022-08-01,SYD,1,33,46.0,BKK,2E,None,None,Bangkok Airways,PG,PG4530
3,2022-08-01,SYD,1,33,46.0,BKK,2E,None,None,Emirates,EK,EK5023
4,2022-08-01,SYD,1,33,46.0,BKK,2E,None,None,Finnair,AY,AY5002


In [35]:
flight_df.to_sql(name='flights', con=engine, if_exists='append', index=False)

DATA SOURCES

-- STATES

"https://en.wikipedia.org/wiki/States_and_territories_of_Australia" Wikipedia 

-- CITY

https://simplemaps.com/data/world-cities  -- JSON file


-- AIRPORTS

https://data.world/ourairports/6fee2ce0-5e8e-4138-a794-f5e9052f38c6/workspace/file?filename=list-of-airports-in-australia-hxl-tags-1.csv

-- TOP ROUTES

https://data.world/tobybellwood/domestic-airlines-top-routes  -- XLSX

-- FLIGHTS 
API